## Find local limit for data size in VRAM
- Repeatedly train models (low iters) with incremented batch sizes
- Find the minimum batch size to give a memory error (so far, 15000 timepoints, 10000 ROIs is fine)
- Test in this notebook and then implement as memory_testing.py

In [1]:
import cebra
import numpy as np
import h5py
import sys

In [7]:
# # global params

# list of all data files
dat_files = ['/media/storage/DATA/lfads_export/f1_221027.h5',
             '/media/storage/DATA/lfads_export/f1_221103.h5',
             '/media/storage/DATA/lfads_export/f2_221103.h5',
             '/media/storage/DATA/lfads_export/f3_221103.h5']

global FILENAME
global TIMESTEPS
global ROIS
global ITERS

FILENAME = dat_files[0]
TIMESTEPS = 15000
ROIS = 10000
ITERS = 1000

In [3]:
# # Define model

cebra_time_model = cebra.CEBRA(
    model_architecture='offset10-model',
    device='cuda_if_available',
    conditional='time',
    temperature_mode='auto',
    min_temperature=0.1,
    time_offsets=10,
    max_iterations=ITERS,                    # kept low for speed
    max_adapt_iterations=500,
    batch_size=None,
    learning_rate=1e-4,
    output_dimension=3,
    verbose=True,
    num_hidden_units=32,
    hybrid=False
    )
print(cebra_time_model)

CEBRA(conditional='time', learning_rate=0.0001, max_iterations=1000,
      model_architecture='offset10-model', output_dimension=3,
      temperature_mode='auto', time_offsets=10, verbose=True)


In [6]:
# # Load data for a single fish

# paths
filename = FILENAME
filename_trunc = filename.split('/')[-1][:-3] # fish and date only
data_folder = 'data/'
filename_dfof = f'{filename[-12:-3]}_dfof.npz'

# choose where in dataset to sample
start, stop = 0, 0+TIMESTEPS

# extract and neural data
# do not attempt to load the entire file 
print("Accessing data...")
with h5py.File(filename, 'r') as f:
    
    # neural 
    neural = f['rois']['dfof']
    print(f"Full neural dataset shape is: {neural.shape}")

    # select first TIMESTEPS timesteps and random ROIS rois
    # neural
    neural_indexes = np.sort(
                        np.random.choice(
                                    np.arange(neural.shape[1]), size=ROIS, replace=False
                                    )
                        )
    neural = np.array(neural[start:stop, neural_indexes])



    print(f"Truncated dataset shapes are:\n \
            neural: {neural.shape}")

    assert(neural.shape == (TIMESTEPS, ROIS))

    # save dataset
    np.savez(f'{data_folder}{filename_dfof}', neural=neural)

    # load dataset
    print("Data accessed.")
    neural = cebra.load_data(f'{data_folder}{filename_dfof}', key="neural")
    print(f"{filename_dfof} loaded.")

Accessing data...
Full neural dataset shape is: (43350, 93122)
Truncated dataset shapes are:
             neural: (15000, 10000)
Data accessed.
f1_221027_dfof.npz loaded.


In [ ]:
# Incrementally load more data and fit models

# paths
filename = FILENAME
filename_trunc = filename.split('/')[-1][:-3] # fish and date only
data_folder = 'data/'
filename_dfof = f'{filename[-12:-3]}_dfof.npz'
filename_output = 'stdout_VRAM_test.txt'

# params
timesteps = 15000

# flags
train_model = False

# redirect print output to text file
orig_stdout = sys.stdout
output = open(filename_output, 'w')
sys.stdout = output


# Incremented timesteps, constant ROIs
while True:
    try:
        # choose where in dataset to sample
        start, stop = 0, 0+timesteps

        # extract and neural data
        # do not attempt to load the entire file 
        print("Accessing data...")
        with h5py.File(filename, 'r') as f:
            
            # neural 
            neural = f['rois']['dfof']
            
            # select first TIMESTEPS timesteps and random ROIS rois
            # neural
            neural_indexes = np.sort(
                                np.random.choice(
                                            np.arange(neural.shape[1]), size=ROIS, replace=False
                                            )
                                )
            neural = np.array(neural[start:stop, neural_indexes])
            print(f"Loaded neural dataset of shape: {neural.shape}")

            assert(neural.shape == (TIMESTEPS, ROIS))

            # save dataset
            np.savez(f'{data_folder}{filename_dfof}', neural=neural)
            print(f"Saved neural dataset of shape: {neural.shape}")


            # load dataset
            neural = cebra.load_data(f'{data_folder}{filename_dfof}', key="neural")
            print(f"Loaded neural dataset of shape {neural.shape}.\n")

            
            # train and save the model
            if train_model:
                model_name = f"{filename_trunc}_time_{timesteps}points_{ROIS}rois_{ITERS}iters.pt"
                model_path = f'models/{model_name}'

                cebra_time_model.fit(neural)
                cebra_time_model.save(model_path)
                print(f"Model fit and saved for neural dataset of shape {neural.shape}.")
    
    except Exception as e:
        print(e)
        f.close()
